In [1]:
import multiprocessing
from multiprocessing.dummy import Pool
from tqdm.auto import tqdm
import imageio
import numpy as np
from glob import glob

In [2]:
n_workers = multiprocessing.cpu_count() // 2

In [3]:
def process_samples(fname):
    fps = imageio.get_reader(fname).get_meta_data()['fps']
    writer = imageio.get_writer(fname.replace('.mp4', '') + '.gif', fps=fps)
    reader = imageio.get_reader(fname)
    with tqdm(total=len(reader), ascii=True) as pbar:
        for i, im in enumerate(reader):
            writer.append_data(im)
            pbar.update(1)
    writer.close()

In [4]:
with Pool(processes=n_workers) as pool:
    fnames = glob('./*.mp4')
    for fname in fnames:
        pool.apply_async(process_samples, (fname,))
    pool.close()
    pool.join()